# Package

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, TransformerConv, GATv2Conv
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges
from sklearn.metrics import roc_auc_score, accuracy_score
import networkx as nx
from tqdm import tqdm
import optuna
import csv
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/Users/cck/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Load the data

In [2]:
base_path = os.getcwd()
node_info_path = os.path.join(base_path, "node_information.csv")
train_path = os.path.join(base_path, "train.txt")
test_path = os.path.join(base_path, "test.txt")

In [3]:
node_features = {}
with open(node_info_path, "r") as f:
    for line in f:
        values = line.strip().split(',') 
        node_id = int(values[0])
        features = np.array([float(x) for x in values[1:]])
        node_features[node_id] = features


In [4]:
# Find the maximum node ID to determine feature matrix size
max_node_id = max(node_features.keys())
feature_size = len(next(iter(node_features.values())))

# Create feature matrix
X = np.zeros((max_node_id + 1, feature_size))
for node_id, features in node_features.items():
    X[node_id] = features

In [5]:
train_edges = []
train_labels = []
with open(train_path , "r") as f:
    for line in f:
        values = line.strip().split()
        node1 = int(values[0])
        node2 = int(values[1])
        label = int(values[2])
        train_edges.append((node1, node2))
        train_labels.append(label)

In [6]:
test_edges = []
with open(test_path , "r") as f:
    for line in f:
        values = line.strip().split()
        node1 = int(values[0])
        node2 = int(values[1])
        test_edges.append((node1, node2))

# PYG Ready Data

In [7]:
# Convert to PyTorch tensors
x = torch.FloatTensor(X)

# Create positive and negative edge indices
pos_edge_index = []
neg_edge_index = []

for edge, label in zip(train_edges, train_labels):
    if label == 1:
        pos_edge_index.append(edge)
    else:
        neg_edge_index.append(edge)

pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long).t()
neg_edge_index = torch.tensor(neg_edge_index, dtype=torch.long).t()

# Create a PyG Data object
data = Data(x=x, num_nodes=max_node_id+1)

# Split positive edges for training and validation
data.train_pos_edge_index = pos_edge_index
data.train_neg_edge_index = neg_edge_index

# Create a combined edge index for message passing
edge_index = torch.cat([pos_edge_index, pos_edge_index.flip(0)], dim=1)  # Make it undirected
data.edge_index = edge_index

data = data.to(device)

# GNN Model Design

## GCN / SAGE / GAT / Transformer

In [ ]:
class GNNLinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, model_type='GCN'):
        super(GNNLinkPredictor, self).__init__()

        # Initialize the convolutional layers
        if model_type == 'GCN':
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, out_channels)

        elif model_type == 'SAGE':
            self.conv1 = SAGEConv(in_channels, hidden_channels)
            self.conv2 = SAGEConv(hidden_channels, out_channels)

        elif model_type == 'GAT':
            self.conv1 = GATConv(in_channels, hidden_channels)
            self.conv2 = GATConv(hidden_channels, out_channels)

        elif model_type == 'GAT2':
            self.conv1 = GATv2Conv(in_channels, hidden_channels)
            self.conv2 = GATv2Conv(hidden_channels, out_channels)

        elif model_type == 'Transformer':
            self.conv1 = TransformerConv(in_channels, hidden_channels)
            self.conv2 = TransformerConv(hidden_channels, out_channels)

        # Initialize the link prediction layer
        self.link_predict = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_channels, 1)
        )

    # encode the input graph data to produce a latent node representation.
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x
    
    # decode the latent node representation to predict whether an edge exists between two nodes.
    def decode(self, z, edge_index):
        # get the source and target node indices
        src, tgt = edge_index
        # combine the node embeddings of the source and target nodes
        return self.link_predict(torch.cat([z[src], z[tgt]], dim=1))

    def forward(self, x, edge_index, predict_edges=None):
        # encode the input graph data
        z = self.encode(x, edge_index)
        
        # If we're given specific edges to predict, only predict those
        if predict_edges is not None:
            return self.decode(z, predict_edges)
        
        # Otherwise, predict all edges
        return self.decode(z, edge_index)


# Training & Validation (Choose the Model (GCN/SAGE/GAT/GAT2/Transformer) Here)

In [ ]:
split_ratio = 0.8
split_size = int(len(train_labels) * split_ratio)

# Validation Data
val_edges = train_edges[split_size:]
val_labels = train_labels[split_size:]
val_edge_index = torch.tensor(val_edges, dtype=torch.long).t().to(device)
val_labels = torch.tensor(val_labels, dtype=torch.float).to(device)
    

# Parameters
in_channels, hidden_channels, out_channels, num_epochs = data.num_features, 128, 64, 100

model_type = 'GCN' # Default: 'GCN", Options: 'SAGE', 'GAT', 'GAT2', 'Transformer'
model = GNNLinkPredictor(in_channels, hidden_channels, out_channels, model_type).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

best_val_auc, best_epoch, patience, counter = 0, 0, 10, 0

In [10]:
def train(model, data, optimizer, batch_size=64):
    model.train()
    
    # Create positive and negative examples
    pos_edge_index = data.train_pos_edge_index
    neg_edge_index = data.train_neg_edge_index
    
    # Shuffle edges
    pos_rand = torch.randperm(pos_edge_index.size(1))
    neg_rand = torch.randperm(neg_edge_index.size(1))
    
    # Split edges into batches
    pos_edge_index = pos_edge_index[:, pos_rand]
    neg_edge_index = neg_edge_index[:, neg_rand]
    
    # Limit to the smaller set's size
    size_min = min(pos_edge_index.size(1), neg_edge_index.size(1))
    pos_edge_index = pos_edge_index[:, :size_min]
    neg_edge_index = neg_edge_index[:, :size_min]
    
    # Process in batches to save memory
    total_loss = 0
    num_batches = (size_min + batch_size - 1) // batch_size
    
    for batch_idx in range(num_batches):
        optimizer.zero_grad() # Clear gradients
        
        # Get batch size
        start = batch_idx * batch_size
        end = min((batch_idx + 1) * batch_size, size_min)
        
        # Create positive and negative batch examples
        pos_batch = pos_edge_index[:, start:end]
        neg_batch = neg_edge_index[:, start:end]
        
        # Create target labels
        pos_y = torch.ones(pos_batch.size(1), 1, device=pos_batch.device)
        neg_y = torch.zeros(neg_batch.size(1), 1, device=neg_batch.device)
        
        # Get predictions
        pos_pred = model(data.x, data.edge_index, pos_batch)
        neg_pred = model(data.x, data.edge_index, neg_batch)
        
        # Compute loss
        loss = F.binary_cross_entropy_with_logits(
            torch.cat([pos_pred, neg_pred], dim=0), # concatenate positive and negative predictions
            torch.cat([pos_y, neg_y], dim=0) # concatenate positive and negative labels
        )
        
        loss.backward() # compute gradients
        optimizer.step() # update weights
        total_loss += loss.item() # Accumulate loss
    
    return total_loss / num_batches


In [11]:
def evaluate(model, data, edge_index, labels):
    model.eval()
    
    with torch.no_grad(): # Do not calculate gradients
        z = model.encode(data.x, data.edge_index) 
        s = model.decode(z, edge_index)
        pred = torch.sigmoid(s).cpu().numpy()
        

        # Calculate AUC
        auc = roc_auc_score(labels, pred)
        
        # Calculate accuracy
        pred_labels = (pred > 0.5).astype(int)
        acc = accuracy_score(labels, pred_labels)
        
    return auc, acc, pred

In [12]:
wd = os.getcwd()
model_path = os.path.join(wd, "best_gnn_model.pt")



print(f"Training {model_type} model...")

for epoch in range(1, num_epochs + 1):
    loss = train(model, data, optimizer)
    
    # Validation
    val_auc, val_acc, _ = evaluate(model, data, val_edge_index, val_labels.cpu().numpy())
    
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}, Val Acc: {val_acc:.4f}')
    
    # Early stopping
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_epoch = epoch
        counter = 0
        torch.save(model.state_dict(), model_path)
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

print(f"Best model at epoch {best_epoch} with validation AUC: {best_val_auc:.4f}")

# Load best model
model.load_state_dict(torch.load(model_path))

Training GCN model...
Epoch: 001, Loss: 0.6322, Val AUC: 0.8138, Val Acc: 0.7338
Epoch: 002, Loss: 0.5381, Val AUC: 0.8424, Val Acc: 0.7533
Epoch: 003, Loss: 0.5161, Val AUC: 0.8505, Val Acc: 0.7590
Epoch: 004, Loss: 0.5045, Val AUC: 0.8564, Val Acc: 0.7710
Epoch: 005, Loss: 0.5003, Val AUC: 0.8665, Val Acc: 0.7714
Epoch: 006, Loss: 0.4883, Val AUC: 0.8688, Val Acc: 0.7786
Epoch: 007, Loss: 0.4789, Val AUC: 0.8754, Val Acc: 0.7833
Epoch: 008, Loss: 0.4616, Val AUC: 0.8848, Val Acc: 0.7933
Epoch: 009, Loss: 0.4460, Val AUC: 0.9008, Val Acc: 0.8224
Epoch: 010, Loss: 0.4220, Val AUC: 0.9236, Val Acc: 0.8638
Epoch: 011, Loss: 0.3886, Val AUC: 0.9368, Val Acc: 0.8795
Epoch: 012, Loss: 0.3632, Val AUC: 0.9425, Val Acc: 0.8981
Epoch: 013, Loss: 0.3579, Val AUC: 0.9435, Val Acc: 0.8919
Epoch: 014, Loss: 0.3414, Val AUC: 0.9463, Val Acc: 0.8857
Epoch: 015, Loss: 0.3381, Val AUC: 0.9501, Val Acc: 0.9048
Epoch: 016, Loss: 0.3319, Val AUC: 0.9528, Val Acc: 0.9043
Epoch: 017, Loss: 0.3241, Val AUC:

/var/folders/lc/t1f6lt2x0xz0nsqj_rj6382m0000gn/T/ipykernel_63870/4045853769.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)

<All keys matched successfully>

# Hyper Parameter Tuning

In [ ]:
def objective(trial, data, device, val_edge_index, val_labels):
    # Model parameters
    hidden_channels = trial.suggest_int('hidden_dim', 32, 256)
    out_channels = trial.suggest_int('out_dim', 16, 128)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    in_channels = data.num_features

    # Model parameters
    model = GNNLinkPredictor(in_channels, hidden_channels, out_channels, model_type="GCN").to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

    # Training loop
    best_val_auc = 0
    # best_epoch = 0
    patience = 10
    counter = 0

    num_epochs = 100
    for epoch in range(1, num_epochs + 1):
        loss = train(model, data, optimizer)

        # Validation
        val_auc, val_acc, _ = evaluate(model, data, val_edge_index, val_labels.cpu().numpy())

        # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_epoch = epoch
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
    return best_val_auc

In [ ]:
# hyperparameter optimization
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, data, device, val_edge_index, val_labels), n_trials=50)

best_params = study.best_params
print(f"Best params: {best_params}")

model = GNNLinkPredictor(data.num_features, best_params['hidden_dim'], best_params['out_dim'], model_type="GCN").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=5e-4)

model_path = os.path.join(base_path, "best_gnn_model.pt")

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    loss = train(model, data, optimizer)

    # Validation
    val_auc, val_acc, _ = evaluate(model, data, val_edge_index, val_labels.cpu().numpy())

    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}, Val Acc: {val_acc:.4f}')

    # Early stopping
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_epoch = epoch
        counter = 0
        torch.save(model.state_dict(), model_path)
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

print(f"Best model at epoch {best_epoch} with validation AUC: {best_val_auc:.4f}")


# Prediction

In [13]:
def predict_test(model, data, test_edges):
    model.eval()

    wd = os.getcwd()
    output_file = os.path.join(wd, "gnn.csv")

    # Convert test edges to tensor
    test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
    
    with torch.no_grad(): # Do not calculate gradients
        # Make predictions
        z = model.encode(data.x, data.edge_index)
        s = model.decode(z, test_edge_index)
        pred_probs = torch.sigmoid(s).cpu().numpy().flatten()
        
        # Get binary predictions
        predictions = (pred_probs > 0.5).astype(int)
        
        # Output predictions
        with open(output_file, "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["ID", "Predicted"])
            for i, pred in enumerate(predictions):
                writer.writerow([i, int(pred)])
        
        print(f"Predictions saved to {output_file}")

In [14]:
predict_test(model, data, test_edges) # predict and output the prediction result to a csv file


Predictions saved to /Users/cck/Desktop/OneDrive_France/DSBA/T2/Machine Learning in Network Science/Kaggle/gnn.csv
